##Pipeline sequencial de Machine Learning para resolver desbalanceamento

*Pipeline sequencial de Machine Learning, você pode dividir o processo em duas etapas principais: (1) o treinamento e previsão do primeiro modelo que distingue as classes majoritárias e marca as instâncias como "majoritária" ou "outras", e (2) o treinamento e execução de um segundo modelo que apenas classifica as instâncias previamente identificadas como "outras"*

## Opções para agrupar as classes minoritárias em "outros":


1ª Distribuição de Frequência:

Calcular a distribuição de cada classe no dataset e definir um critério baseado na porcentagem de ocorrência. Por exemplo, classes que representam menos de 5% das amostras totais podem ser consideradas para agrupar em "outras".

In [ ]:
import pandas as pd

# Exemplo de dataset com a coluna 'classe'
dados = {'classe': ['A', 'A', 'B', 'B', 'B', 'C', 'C', 'D', 'E', 'E', 'F']}
df = pd.DataFrame(dados)

# Calcula a frequência de cada classe
frequencias = df['classe'].value_counts(normalize=True) * 100

# Define um limiar para classes minoritárias (por exemplo, < 5%)
limiar = 5
classes_minoritaras = frequencias[frequencias < limiar].index

print("Classes minoritárias: ", list(classes_minoritaras))


2ª Avaliação de Métricas (Precision, Recall e F1-Score):

Classes com F1-Score baixo (indicando dificuldade de aprendizado pelo modelo) são candidatas ideais para serem agrupadas:

In [ ]:
from sklearn.metrics import classification_report

# Exemplo de rótulos reais e previstos
y_true = ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'E', 'E', 'F']
y_pred = ['A', 'B', 'B', 'B', 'A', 'C', 'A', 'E', 'D', 'A']

# Gera o relatório de classificação
relatorio = classification_report(y_true, y_pred, output_dict=True)

# Identifica classes com F1-Score abaixo de 0.5
classes_minoritaras = [classe for classe, metricas in relatorio.items()
                       if isinstance(metricas, dict) and metricas['f1-score'] < 0.5]

print("Classes com baixo desempenho: ", classes_minoritaras)


##Passo a passo do Pipeline Sequencial:



*   Primeiro Modelo: Classifica as instâncias entre "classes majoritárias" e "outras";
*   Segundo Modelo: Refina a previsão para classificar as observações que foram previamente rotuladas como "outras".




In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report

# Exemplo de dataset (Substituir pelos dados reais)
X = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'feature2': [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
})
y = pd.Series(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'E', 'E', 'F'])

# Escolhendo pela frequência e identifica as classes minoritárias:
frequencias = y.value_counts(normalize=True) * 100
limiar = 5  # Define o limiar para classes minoritárias
classes_minoritaras = frequencias[frequencias < limiar].index

# Agrupa as classes minoritárias na categoria "outras"
y_agrupado = np.where(y.isin(classes_minoritaras), 'outras', y)

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_agrupado, test_size=0.3, random_state=42)

# Modelo 1: Classificação entre majoritárias e "outras"
model_1 = RandomForestClassifier(random_state=42)

# Etapa intermediária: Filtro para passar apenas as instâncias "outras" para o próximo modelo
class FiltrarOutras(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y):
        # Retorna apenas as instâncias rotuladas como "outras"
        mask = (y == 'outras')
        return X[mask], y[mask]

# Modelo 2: Classificação entre as classes minoritárias no grupo "outras"
model_2 = RandomForestClassifier(random_state=42)

# Pipeline completo
class PipelineComposto(BaseEstimator):
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2

    def fit(self, X, y):
        # Treina o primeiro modelo
        self.model_1.fit(X, y)

        # Identifica as instâncias "outras"
        outras_mask = (self.model_1.predict(X) == 'outras')
        X_outras, y_outras = X[outras_mask], y[outras_mask]

        # Treina o segundo modelo apenas com as instâncias "outras"
        if len(X_outras) > 0:
            self.model_2.fit(X_outras, y_outras)
        return self

    def predict(self, X):
        # Primeira previsão: Majoritária vs Outras
        pred_1 = self.model_1.predict(X)

        # Segunda previsão: Classificação das instâncias "outras"
        outras_mask = (pred_1 == 'outras')
        if outras_mask.any():
            pred_outras = self.model_2.predict(X[outras_mask])
            pred_1[outras_mask] = pred_outras

        return pred_1

# Cria e treina o pipeline
pipeline = PipelineComposto(model_1=model_1, model_2=model_2)
pipeline.fit(X_train, y_train)

# Faz previsões no conjunto de teste
y_pred = pipeline.predict(X_test)

# Avaliação do modelo
print(classification_report(y_test, y_pred))
